In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/reid/deep-person-reid
!python setup.py develop
import torch
from torch import nn
import torchreid
%cd ..

/content/drive/MyDrive/reid/deep-person-reid
running develop
running egg_info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'torchreid.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.7/torchreid/metrics/rank_cylib/rank_cy.cpython-37m-x86_64-linux-gnu.so -> torchreid/metrics/rank_cylib
Creating /usr/local/lib/python3.7/dist-packages/torchreid.egg-link (link to .)
Adding torchreid 1.4.0 to easy-install.pth file

Installed /content/drive/MyDrive/reid/deep-person-reid
Processing dependencies for torchreid==1.4.0
Searching for isort==4.3.21
Reading https://pypi.org/simple/isort/
Best match: isort 4.3.21
Processing isort-4.3.21-py2.py3-none-any.whl
Installing isort-4.3.21-py2.py3-none-any.whl to /usr/local/lib/python3.7/dist-packages
Adding isor

In [3]:
datamanager = torchreid.data.datamanager.ImageDataManager(
    root='',
    sources='dukemtmcreid',
    height=256,
    width=128,
    transforms='random_flip',
    batch_size_train=64,
    batch_size_test=64
)
# return train loader of source data
train_loader = datamanager.train_loader

# return test loader of target data
test_loader = datamanager.test_loader

# return train loader of target data
train_loader_t = datamanager.train_loader_t


Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------
=> Loading test (target) dataset


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------


  **************** Summary ****************
  source            : ['dukemtmcreid']
  # source datasets : 1
  # source ids      : 702
  # source images   : 16522
  # source cameras  : 8
  target            : ['dukemtmcreid']
  *****************************************




In [41]:
for target in list(test_loader.keys()):

  print(test_loader[target]['gallery'])
print(test_loader.values)

<built-in method values of dict object at 0x7f738c9e5190>


In [ ]:
model = torchreid.models.osnet_x1_0(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model = model.cuda()

weight_path = 'pretrained/osnet_x1_0_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'
torchreid.utils.torchtools.load_pretrained_weights(model, weight_path)
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0015
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "pretrained/osnet_x1_0_duke_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"


In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler
)
engine.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=10,
    test_only=False,
    save_dir='log/osnet_x1_0_dukemtmcreid',
    print_freq=10
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/10][10/258]	time 0.911 (1.075)	data 0.001 (0.143)	eta 0:46:03	loss 6.1969 (6.3562)	acc 9.3750 (9.6875)	lr 0.001500
epoch: [1/10][20/258]	time 0.908 (0.991)	data 0.001 (0.072)	eta 0:42:17	loss 6.1192 (6.2449)	acc 12.5000 (10.3125)	lr 0.001500
epoch: [1/10][30/258]	time 0.911 (0.964)	data 0.001 (0.048)	eta 0:40:57	loss 5.6831 (6.1175)	acc 10.9375 (10.7292)	lr 0.001500
epoch: [1/10][40/258]	time 0.908 (0.951)	data 0.001 (0.036)	eta 0:40:15	loss 5.0742 (5.9098)	acc 23.4375 (12.7734)	lr 0.001500
epoch: [1/10][50/258]	time 0.916 (0.944)	data 0.001 (0.029)	eta 0:39:48	loss 4.9128 (5.6846)	acc 15.6250 (15.3750)	lr 0.001500
epoch: [1/10][60/258]	time 0.915 (0.939)	data 0.000 (0.024)	eta 0:39:27	loss 4.4704 (5.4864)	acc 28.1250 (17.1875)	lr 0.001500
epoch: [1/10][70/258]	time 0.918 (0.936)	data 0.001 (0.021)	eta 0:39:10	loss 4.3592 (5.3007)	acc 29.6875 (19.5312)	lr 0.001500
epoch: [1/10][80/258]	time 0.916 (0.934)	data 0.001 (0.019)	eta 0:38:55	loss 4.5072 (5.1387)	acc 31.2500 (22.0117

In [4]:

model1 = torchreid.models.osnet_x1_0(
    num_classes=datamanager.num_train_pids,
    pretrained=False,
    loss='softmax'
)

model1 = model1.cuda()

weight_path = 'log/osnet_x1_0_dukemtmcreid/model/model.pth.tar-10'
torchreid.utils.torchtools.load_pretrained_weights(model1, weight_path)
optimizer = torchreid.optim.build_optimizer(
    model1, optim='adam', lr=0.0015
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=5
)

Successfully loaded pretrained weights from "log/osnet_x1_0_dukemtmcreid/model/model.pth.tar-10"


In [6]:
engine1 = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model1, optimizer, scheduler=scheduler
)
engine1.run(
    max_epoch=10,
    start_eval=0,
    eval_freq=1,
    test_only=True,
    visrank=True,
    visrank_topk=5,
    save_dir='log/osnet_x1_0_dukemtmcreid',
    print_freq=10
)

##### Evaluating dukemtmcreid (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Done, obtained 2228-by-512 matrix
Extracting features from gallery set ...
Done, obtained 17661-by-512 matrix
Speed: 0.0320 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 63.9%
CMC curve
Rank-1  : 83.4%
Rank-5  : 91.7%
Rank-10 : 93.2%
Rank-20 : 95.0%
# query: 2228
# gallery 17661
Visualizing top-5 ranks ...
- done 100/2228
- done 200/2228
- done 300/2228
- done 400/2228
- done 500/2228
- done 600/2228
- done 700/2228
- done 800/2228
- done 900/2228
- done 1000/2228
- done 1100/2228
- done 1200/2228
- done 1300/2228
- done 1400/2228
- done 1500/2228
- done 1600/2228
- done 1700/2228
- done 1800/2228
- done 1900/2228
- done 2000/2228
- done 2100/2228
- done 2200/2228
Done. Images have been saved to "log/osnet_x1_0_dukemtmcreid/visrank_dukemtmcreid" ...


In [44]:
!python deep-person-reid/tools/visualize_actmap.py --root ./ -d dukemtmcreid -m osnet_x1_0 --weights log/osnet_x1_0_dukemtmcreid/model/model.pth.tar-10 --save-dir log/visactmap_osnet_x1_0_dukemtmcreid_test

Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded DukeMTMCreID
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader 